In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = input("Enter your APImetrics API key for the intended project: ")

In [ ]:
from apimetrics_api import APImetricsAPI 

# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

def find_header(header_key, call):
    header_key = header_key.lower()
    return next(
        (header for header in call['request']['headers'] if header['key'].lower() == header_key),
        None,
    )

def find_condition(source, variable_path, conditions):
    return next(
        (condition for condition in conditions if condition['source'] == source and condition['variable_path'].lower() == variable_path.lower()),
        None
    )

HSBC = False
RBS = True

calls = CLIENT.get_calls()
for call in calls['results']:
    if "api_expert:exclude" not in call['meta']['tags']:
        
        print(f"API: {call['meta']['name']}...", end='')
        needs_save = False
        conditions = CLIENT.get_call_conditions(call['id'])
        cond_array = conditions['conditions']
        
        if 'ob_v:3.1' in call['meta']['tags']:
            has_header = find_header("x-fapi-interaction-id", call)
            if not has_header:
                needs_save = True
                request = call['request']
                headers = request['headers'] or []
                headers.append(dict(key='x-fapi-interaction-id', value='%%GUID%%'))
                CLIENT.update_call(call['id'], dict(request=dict(headers=headers)))
                print(' ! ', end='')
            else:
                print(' . ', end='')

            has_cond = find_condition('HEADERS', 'x-fapi-interaction-id', cond_array)
            if not has_cond:
                cond_array.append({
                    'source': 'HEADERS',
                    'variable_path': 'x-fapi-interaction-id',
                    'condition': 'EXISTS',
                    'val': '',
                    'test_result_on_true': None,
                    'test_result_on_false': 'CONTENT_ERROR',
                    'variable_name': 'X_FAPI_INTERACTION_ID',
                })
                needs_save = True
                print(' O', end='')
            else:
                print(' o', end='')

        if RBS:
            has_cond = find_condition('HEADERS', 'X-Rbs-Interaction-Id', cond_array)
            if not has_cond:
                cond_array.append({
                    'source': 'HEADERS',
                    'variable_path': 'X-Rbs-Interaction-Id',
                    'condition': 'EXISTS',
                    'val': '',
                    'test_result_on_true': None,
                    'test_result_on_false': None,
                    'variable_name': 'X_RBS_INTERACTION_ID',
                })
                needs_save = True
                print(' O', end='')
            else:
                print(' o', end='')

        if HSBC:
            has_cond = find_condition('HEADERS', 'x-amzn-RequestId', cond_array)
            if not has_cond:
                cond_array.append({
                    'source': 'HEADERS',
                    'variable_path': 'x-amzn-RequestId',
                    'condition': 'EXISTS',
                    'val': '',
                    'test_result_on_true': None,
                    'test_result_on_false': None,
                    'variable_name': 'X_AMZN_REQUEST_ID',
                })
                needs_save = True
                print(' O', end='')
            else:
                print(' o', end='')


            has_cond = find_condition('HEADERS', 'x-amz-apigw-id', cond_array)
            if not has_cond:
                cond_array.append({
                    'source': 'HEADERS',
                    'variable_path': 'x-amz-apigw-id',
                    'condition': 'EXISTS',
                    'val': '',
                    'test_result_on_true': None,
                    'test_result_on_false': None,
                    'variable_name': 'X_AMZ_APIGW_ID',
                })
                needs_save = True
                print(' O', end='')
            else:
                print(' o', end='')

            has_cond = find_condition('HEADERS', 'X-Amzn-Trace-Id', cond_array)
            if not has_cond:
                cond_array.append({
                    'source': 'HEADERS',
                    'variable_path': 'X-Amzn-Trace-Id',
                    'condition': 'EXISTS',
                    'val': '',
                    'test_result_on_true': None,
                    'test_result_on_false': None,
                    'variable_name': 'X_AMZN_TRACE_ID',
                })
                needs_save = True
                print(' O', end='')
            else:
                print(' o', end='')

        if needs_save:
            CLIENT.set_call_conditions(call['id'], dict(conditions=cond_array))
            print(' !')
        else:
            print(' .')